In [1]:
import numpy as np
import re
import excel_function_errors.basic_operation_errors as basic
import excel_function_errors.math_function_errors as math
import excel_function_errors.function_evaluators as evaluator

In [2]:
A1 = 5
A2 = 8
A3 = 7

B1 = 3
B2 = 4
B3 = 2

A1_ERROR = 2
A2_ERROR = 3
A3_ERROR = 4

B1_ERROR = 1
B2_ERROR = 4
B3_ERROR = 1


value_error_dict = {
    "A1": A1_ERROR,
    "A2": A2_ERROR,
    "A3": A3_ERROR, 
    "B1": B1_ERROR,
    "B2": B2_ERROR,
    "B3": B3_ERROR
}

value_value_dict = {
    "A1": A1,
    "A2": A2,
    "A3": A3, 
    "B1": B1,
    "B2": B2,
    "B3": B3
}

In [3]:
test1 = "A1 ^ A2 + A3 * B1 / B2 ^ B3"
expression = test1
expression_stripped = re.sub(r"\s", "", expression)
expression_vars = re.split(r"\+|\-|\*|\/|\^", expression_stripped)
expression_operators = re.findall(r"\+|\-|\*|\/|\^", expression_stripped)

In [4]:
def expression_parser(variables, operators):
    """
    Takes expression without parentheses and returns value
    
    Args:
    variables: list of variables of size n
    operators: list of operators of size n - 1
    
    Returns:
    final_value: result of expresion
    final_error: error of expression
    """
    
    expression_vars_func = variables.copy()
    expression_operators_func = operators.copy()
    
    # Order of operations
    PEMDAS = "^/*-+"
    
    # For each operator type
    for operator_type in PEMDAS:
        
        # While that operator type is still in the list
        while (operator_type in expression_operators_func):
            
            # Take index of operator
            i = expression_operators_func.index(operator_type)

            # Find out if a,b are numbers, variables, or excel expressions
            a_variable = expression_vars_func[i]
            b_variable = expression_vars_func[i+1]
            
            a_type = variable_type_evaluator(a_variable)
            b_type = variable_type_evaluator(b_variable)

            # Check to see if variable has already been computed. 
            # If not, check to see if numeric, variable, or exel expression
            if a_variable in value_error_dict:
                a = value_value_dict[a_variable]
                a_error = value_error_dict[a_variable]   
            elif a_type == "numeric":
                a = float(a_variable)
                a_error = 0
            elif a_type == "variable":
                a = value_value_dict[a_variable]
                a_error = value_error_dict[a_variable]
            elif a_type == "excel_expression":
                a, a_error = evaluator.excel_expression_evaluator(a_variable)
            
            
            if b_variable in value_error_dict:
                b = value_value_dict[b_variable]
                b_error = value_error_dict[b_variable] 
            elif b_type == "numeric":
                b = float(b_variable)
                b_error = 0
            elif b_type == "variable":
                b = value_value_dict[b_variable]
                b_error = value_error_dict[b_variable]
            elif b_type == "excel_expression":
                b, b_error = evaluator.excel_expression_evaluator(b_variable)

            
            print(a_type, b_type)
            # Combine the names of the variables
            new_var_name = a_variable + operator_type + b_variable
            
            # Evaluate the value of the operation
            value_value_dict[new_var_name] = evaluator.value_evaluator(a,b,operator_type)
            
            # Evaluate the error of the operation
            value_error_dict[new_var_name] = evaluator.error_evaluator(a,b,a_error, b_error, operator_type)

            # Delete the operation from the list
            del expression_operators_func[i]
            del expression_vars_func[i]
            
            # Save the value and the list
            expression_vars_func[i] = new_var_name
    
    final_value = value_value_dict[new_var_name]
    final_error = value_error_dict[new_var_name]
    return final_value, final_error

In [5]:
def variable_type_evaluator(var):
    """
    Returns if a variable is numeric, a variable, or an excel expression
    
    Args:
    var: String that is a number, a variable, or an expression expression
    
    Returns:
    "numeric", "variable", "excel_expression"
    
    """
    if var.isnumeric(): return "numeric"
    elif "(" not in var: return "variable"
    else: return "excel_expression"

In [9]:
simple_test = "SUM(A1:A5) + SINH(1,3) "
expression = simple_test
expression_stripped = re.sub(r"\s", "", expression)
expression_vars = re.split(r"\+|\-|\*|\/|\^", expression_stripped)
expression_operators = re.findall(r"\+|\-|\*|\/|\^", expression_stripped)
expression_parser(expression_vars, expression_operators)

excel_expression excel_expression


(24, 74.3042156277264)

In [45]:
def expression_evaluator(expression): 
    from itertools import islice
    
    # If no parentheses in the expression, evaluate.
    if "(" not in expression:
        expression_vars = re.split(r"\+|\-|\*|\/|\^", expression)
        expression_operators = re.findall(r"\+|\-|\*|\/|\^", expression)
        return expression_vars, expression_operators
    
    stack = []
    expression_vars = []
    expression_operators = []
    
    # Iterate through every character of the expression
    expression_iter = iter(range(len(expression)))
    for i in expression_iter:
        # Add "(" to the stack
        if expression[i] == "(":
            stack.append(i)
        
        # Pop ")" from the stack
        if expression[i] == ")":
            beginning = stack.pop()
            
            # If the stack is empty, then that is an expression that can be evaluated
            if len(stack) == 0:
                expression_vars.append(expression[beginning+1:i])
        
        # If stack is empty, find rest of expression until "(" to evaluate
        elif len(stack) == 0:
            beginning = i
            expression_remaining = expression[i:]
            
            # If no parentheses left in the expression, calculate
            if "(" not in expression_remaining:
                expression_vars.extend(re.split(r"\+|\-|\*|\/|\^", expression_remaining)[1:])
                expression_operators.extend(re.findall(r"\+|\-|\*|\/|\^", expression_remaining))
                break
                
            # If there is a parenthesis in the expression
            else:
                end = expression_remaining.index("(")
                expression_without_parenthesis = expression[i:i+end]
                expression_vars.extend(re.split(r"\+|\-|\*|\/|\^", expression_without_parenthesis))
                expression_operators.extend(re.findall(r"\+|\-|\*|\/|\^", expression_without_parenthesis))
                if end == 1:
                    pass
                if end > 1:
                    next(islice(expression_iter,end-1,end), None)
                
    expression_vars = list(filter(lambda x: x != "", expression_vars))
    expression_operators = list(filter(lambda x: x != "", expression_operators))

    return expression_vars,expression_operators

In [22]:
test2 = "((([A1 ^ A2] + SUM(A1:B2) + (A3 * B1)) / B2) ^ B3)"
test2 = re.sub(r"\s", "", test2)
temp = re.sub(r"\{|\[", "(", test2)
test2 = re.sub(r"\]|\}", ")", temp)
test2

'((((A1^A2)+SUM(A1:B2)+(A3*B1))/B2)^B3)'

In [23]:
stack = []
for index, character in enumerate(test2):
    if character == "(":
        stack.append(index)
    if character == ")":
        expression_beginning = stack.pop()
        mini_expression = test2[expression_beginning+1:index]  
        print(mini_expression)
        #value_value_dict[mini_expression],value_error_dict[mini_expression] =  \
        #expression_parser(*expression_evaluator(mini_expression))

#value_value_dict[test2[1:-1]], value_error_dict[test2[1:-1]], 

A1^A2
A1:B2
A3*B1
(A1^A2)+SUM(A1:B2)+(A3*B1)
((A1^A2)+SUM(A1:B2)+(A3*B1))/B2
(((A1^A2)+SUM(A1:B2)+(A3*B1))/B2)^B3


In [31]:
import parser
eval("((((A1 ** A2) + (A3 * B1)) / B2) ** B3)")

9537768582.25

# Next Steps

1. Check to make sure code works with difficult examples
2. Streamline pipeline
3. Program in edge case =A1
4. Program in variable + constants in expressions
5. Program in variable + constants + excel functionsin expressions

In [ ]:
types_of_inputs = ["(a, a_error)", "(a, b, a_error, b_error)", "(values, errors)",
                   "(other)", "(a_error, base)", "()"]